In [1]:
import pandas as pd

test = pd.read_csv("./data/train.csv")

/home/jcapela/miniconda3/envs/plants_sm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
test_reduced = test.iloc[:5000]
test_reduced.to_csv("./data/test_reduced.csv", index=False)

In [1]:
import pandas as pd

test = pd.read_csv("./data/esm2_t12_35m/test_esm2_t12_35M_UR50D.csv")

/home/jcapela/miniconda3/envs/plants_sm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
test.iloc[:, 2780:].columns

Index(['ESM_esm2_t12_35M_UR50D_0', 'ESM_esm2_t12_35M_UR50D_1',
       'ESM_esm2_t12_35M_UR50D_2', 'ESM_esm2_t12_35M_UR50D_3',
       'ESM_esm2_t12_35M_UR50D_4', 'ESM_esm2_t12_35M_UR50D_5',
       'ESM_esm2_t12_35M_UR50D_6', 'ESM_esm2_t12_35M_UR50D_7',
       'ESM_esm2_t12_35M_UR50D_8', 'ESM_esm2_t12_35M_UR50D_9',
       ...
       'ESM_esm2_t12_35M_UR50D_470', 'ESM_esm2_t12_35M_UR50D_471',
       'ESM_esm2_t12_35M_UR50D_472', 'ESM_esm2_t12_35M_UR50D_473',
       'ESM_esm2_t12_35M_UR50D_474', 'ESM_esm2_t12_35M_UR50D_475',
       'ESM_esm2_t12_35M_UR50D_476', 'ESM_esm2_t12_35M_UR50D_477',
       'ESM_esm2_t12_35M_UR50D_478', 'ESM_esm2_t12_35M_UR50D_479'],
      dtype='object', length=480)

In [16]:
test.iloc[:, 8:2779].columns

Index(['1', '2', '3', '4', '5', '6', '7', '1.1', '1.10', '1.11',
       ...
       '7.4.2.5', '7.4.2.8', '7.5.2.11', '7.6.2.1', '7.6.2.11', '7.6.2.13',
       '7.6.2.2', '7.6.2.3', '7.6.2.5', '7.6.2.8'],
      dtype='object', length=2771)

In [14]:
test_reduced = test[test.sequence.str.len() <= test.sequence.str.len().quantile(0.99)]
test_reduced = test_reduced[test_reduced.sequence.str.len() >= test_reduced.sequence.str.len().quantile(0.01)]
test_reduced.to_csv("./data/test_reduced.csv", index=False)

In [1]:
from plants_sm.data_structures.dataset.single_input_dataset import SingleInputDataset


dataset = SingleInputDataset.from_csv("./data/test_reduced.csv", instances_ids_field="accession", representation_field="sequence", 
                                labels_field=slice(8, -1))

/home/jcapela/PlantsSM/src/plants_sm/io/csv.py:132: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  df = reader.read()


In [3]:
from plants_sm.data_standardization.truncation import Truncator
Truncator(max_length=1397).fit_transform(dataset)

Truncator: 100%|██████████| 5000/5000 [00:00<00:00, 39455.16it/s]


In [6]:
max_value = 0
for instance in dataset.get_instances().values():
    length = len(instance)
    if length > max_value:
        max_value = length
print(max_value)

1397


In [2]:


from plants_sm.featurization.proteins.bio_embeddings.esm import ESMEncoder


ESMEncoder(device="cuda", esm_function="esm2_t36_3B_UR50D", batch_size=1, num_gpus=4).fit_transform(dataset)

/home/jcapela/miniconda3/envs/plants_sm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [5]:
from plants_sm.featurization.proteins.bio_embeddings.word2vec import Word2Vec


Word2Vec().fit_transform(dataset)

Word2Vec: 100%|██████████| 5000/5000 [00:06<00:00, 824.12it/s]


In [28]:
len(dataset.features_fields["place_holder"])

640

In [23]:
dataset.features["place_holder"]["I3R176"].shape

(320,)

In [6]:
dataset.to_csv("./data/test_reduced_esm.csv")

In [9]:
import pandas as pd
pd.read_csv("./data/test_reduced_esm.csv")

/home/jcapela/miniconda3/envs/plants_sm/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,accession,name,EC,EC1,EC2,EC3,EC4,1,2,...,word2vec_503,word2vec_504,word2vec_505,word2vec_506,word2vec_507,word2vec_508,word2vec_509,word2vec_510,word2vec_511,word2vec_512
0,0,Q7XQ85,1A12_ORYSJ,4.4.1.14,4,4.4,4.4.1,4.4.1.14,0.0,0.0,...,0.019150,-0.018458,0.062411,-0.029814,-0.065052,-0.026976,-0.013128,-0.059176,0.035541,-0.051473
1,1,A0A0P0WIY3,1A13_ORYSJ,4.4.1.14,4,4.4,4.4.1,4.4.1.14,0.0,0.0,...,0.015985,-0.020694,0.061179,-0.028515,-0.063737,-0.033109,-0.017545,-0.059569,0.035243,-0.049128
2,2,Q43309,1A14_ARATH,4.4.1.14,4,4.4,4.4.1,4.4.1.14,0.0,0.0,...,0.016950,-0.014905,0.070020,-0.032560,-0.068296,-0.024372,-0.017000,-0.061191,0.040713,-0.054937
3,3,P29535,1A14_SOLLC,4.4.1.14,4,4.4,4.4.1,4.4.1.14,0.0,0.0,...,0.017325,-0.015279,0.069340,-0.032585,-0.063828,-0.023135,-0.020652,-0.061863,0.039154,-0.051963
4,4,A0A0P0UZP7,1A15_ORYSJ,4.4.1.14,4,4.4,4.4.1,4.4.1.14,0.0,0.0,...,0.014068,-0.018709,0.063375,-0.030802,-0.062235,-0.027905,-0.015503,-0.061831,0.039371,-0.047659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,Q6F854,PANB_ACIAD,2.1.2.11,2,2.1,2.1.2,2.1.2.11,0.0,1.0,...,0.013835,-0.021179,0.066692,-0.032096,-0.062312,-0.025367,-0.014526,-0.064466,0.037195,-0.054333
4996,4996,A8ALE9,PANB_CITK8,2.1.2.11,2,2.1,2.1.2,2.1.2.11,0.0,1.0,...,0.014172,-0.023649,0.065384,-0.031154,-0.060188,-0.029458,-0.015860,-0.064528,0.034582,-0.053859
4997,4997,A5N5W0,PANB_CLOK5,2.1.2.11,2,2.1,2.1.2,2.1.2.11,0.0,1.0,...,0.018737,-0.014015,0.072756,-0.034109,-0.061583,-0.023594,-0.023227,-0.072531,0.035952,-0.054019
4998,4998,A0PXQ3,PANB_CLONN,2.1.2.11,2,2.1,2.1.2,2.1.2.11,0.0,1.0,...,0.015429,-0.015810,0.072191,-0.031280,-0.061021,-0.021622,-0.023619,-0.072106,0.035115,-0.054017
